<a href="https://colab.research.google.com/github/allice-lliu/Movie-Script-Project/blob/main/Bee_Movie1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
#path_to_file = tf.keras.utils.get_file('script_1.txt', '/content/drive/My Drive/script_1.txt')
path_to_file = '/content/drive/My Drive/Bee Movie Project/script_1.txt'
print(path_to_file)

/content/drive/My Drive/Bee Movie Project/script_1.txt


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 50139 characters


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

69 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  '0' :   7,
  '1' :   8,
  '2' :   9,
  '3' :  10,
  '4' :  11,
  '5' :  12,
  '6' :  13,
  '7' :  14,
  '8' :  15,
  '9' :  16,
  ':' :  17,
  '?' :  18,
  'A' :  19,
  ...
}


In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
A
c
c
o


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeffAccording to all known laws of aviation, there is no way a bee should be able to fly. Its wings are '
'too small to get its fat little body off the ground. The bee, of course, flies anyway because bees do'
"n't care what humans think is impossible. Yellow, black. Yellow, black. Yellow, black. Yellow, black."
" Ooh, black and yellow! Let's shake it up a little. Barry! Breakfast is ready! Ooming! Hang on a seco"
"nd. Hello? - Barry? - Adam? - Oan you believe this is happening? - I can't. I'll pick you up. Looking"


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeffAccording to all known laws of aviation, there is no way a bee should be able to fly. Its wings are'
Target data: 'According to all known laws of aviation, there is no way a bee should be able to fly. Its wings are '


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 68 ('\ufeff')
  expected output: 19 ('A')
Step    1
  input: 19 ('A')
  expected output: 44 ('c')
Step    2
  input: 44 ('c')
  expected output: 44 ('c')
Step    3
  input: 44 ('c')
  expected output: 56 ('o')
Step    4
  input: 56 ('o')
  expected output: 59 ('r')


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 69) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           17664     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 69)            70725     
Total params: 4,026,693
Trainable params: 4,026,693
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([40, 12,  0, 54, 54, 62, 68,  7, 21, 37,  0,  2, 59, 10, 49,  7, 19,
        1, 24, 44,  3, 42, 39,  7,  5, 46, 64,  5, 24,  7, 38, 22, 58, 66,
       26, 10, 16, 40, 15, 33, 32,  9, 37, 52, 29, 18, 45, 56, 37, 64, 14,
       32, 38, 34, 36, 33, 22, 54, 20, 60, 39, 11, 39, 27,  1, 50, 46, 44,
       54, 66, 19, 53, 31, 22,  1, 45, 43, 57, 21, 29, 29, 32, 25, 33, 11,
       54,  7, 33, 68, 43, 66, 18, 41, 34, 51, 58, 63, 50, 18, 42])

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "! Hold it, Your Honor! You want a smoking gun? Here is your smoking gun. What is that? It's a bee sm"

Next Char Predictions: 
 'W5 mmu\ufeff0DT "r3h0A!Gc\'aV0-ew-G0UEqyI39W8PO2TkL?doTw7OUQSPEmBsV4VJ!iecmyAlNE!dbpDLLOHP4m0P\ufeffby?YQjqvi?a'


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 69)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.232881


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=400, callbacks=[checkpoint_callback])

Epoch 1/400
7/7 [==============================] - 1s 140ms/step - loss: 4.7048
Epoch 2/400
7/7 [==============================] - 1s 153ms/step - loss: 3.8303
Epoch 3/400
7/7 [==============================] - 1s 151ms/step - loss: 3.5242
Epoch 4/400
7/7 [==============================] - 1s 128ms/step - loss: 3.1566
Epoch 5/400
7/7 [==============================] - 1s 160ms/step - loss: 3.0131
Epoch 6/400
7/7 [==============================] - 1s 126ms/step - loss: 2.8937
Epoch 7/400
7/7 [==============================] - 1s 159ms/step - loss: 2.7396
Epoch 8/400
7/7 [==============================] - 1s 152ms/step - loss: 2.6007
Epoch 9/400
7/7 [==============================] - 1s 126ms/step - loss: 2.4999
Epoch 10/400
7/7 [==============================] - 1s 160ms/step - loss: 2.4291
Epoch 11/400
7/7 [==============================] - 1s 152ms/step - loss: 2.3780
Epoch 12/400
7/7 [==============================] - 1s 169ms/step - loss: 2.3375
Epoch 13/400
7/7 [===================

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            17664     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 69)             70725     
Total params: 4,026,693
Trainable params: 4,026,693
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=" "))

 Ohappecentrath dake down! Pinhead. - Oheck out the new smoker. - Oh, sweet. That's the one you want. The Thomas 3000 Oh, my. Oould you get a nurse to close that window? - Why? - The smoke. Bet some on the riadecentrathe als? Restroom attendant's open, not for the reason you think. - Any chance of getting the Krelman? - Sut! She saved my life. I gotta say something. All right, here it goes. Nah. What would I say? I could re were friends. The last thing we want to do is upset bees! You're too late! It's ours now! You, sir, will be line! Thinking bee! Thinking bee! Thinking bee! - Vanessa, aim, varry, she have. I could say anything right now. I'm gonna get antennas inside the tram at all times. - Wonder what it'll be like? - A little scary. Welcome to Honey Farms. I am onto something huge here. I'm going to Alaska. Moose blood, crazy stuff. How those flowers! Wow! I'm out! I can't believe I'm on times? "The surflearever? That's an insane choice to have to make. I'm relieved. Now we only 